In [ ]:
import numpy as np
import plotly.graph_objects as go

from sklearn.linear_model import LinearRegression

# Model misspecification: Figs. S13 & S14

## 1. Wrong lag, no latent confounder

$X_t =b(t)X_{t−1}+η_t$ \
​
$Y_t=a(t)Y{t−1} +cX_{t−τ_{true}} +γ_t$

Estimate: $Y_t​=αY_{t−1}​+βX_{t−τ_{assumed}}​+ε_t​$

In [ ]:
# -- Simulation function
def simulate_time_varying_tau(a0, a_end, b0, b_end, c, sigma_eta2, sigma_gamma2,
                          tau_true, N, burn=500, seed=1):
    """
    Simulate coupled AR(1) processes with slowly time-varying coefficients:
      X_t = b_t X_{t-1} + eta_t
      Y_t = a_t Y_{t-1} + c X_{t - tau_true} + gamma_t
    Returns:
        X[-N:], Y[-N:], a_t[-N:], b_t[-N:]  (all length N)
    """
    rng = np.random.default_rng(seed)
    total = N + burn + abs(tau_true)
    # desired slope on the returned N-block
    slope_a = (a_end - a0) / float(max(1, N - 1))
    slope_b = (b_end - b0) / float(max(1, N - 1))

    # build last-N linear sequences (these are the "true" returned sequences)
    a_ret = a0 + slope_a * np.arange(N)    # length N: from a0 -> a_end
    b_ret = b0 + slope_b * np.arange(N)    # length N: from b0 -> b_end

    # Prepend earlier values by linear extrapolation backwards so full array length == total
    pre_len = total - N
    if pre_len > 0:
        # for indices 0 .. pre_len-1, set values earlier than a0 by stepping back
        # so that a_ret[0] == a0 at position index pre_len in the full array
        a_pre = a0 - slope_a * np.arange(pre_len, 0, -1)  # length pre_len
        b_pre = b0 - slope_b * np.arange(pre_len, 0, -1)
        a_t = np.concatenate([a_pre, a_ret])
        b_t = np.concatenate([b_pre, b_ret])
    else:
        a_t = a_ret.copy()
        b_t = b_ret.copy()

    # safety: ensure lengths are correct
    assert len(a_t) == total
    assert len(b_t) == total

    # allocate and simulate
    X = np.zeros(total)
    Y = np.zeros(total)
    eta = rng.normal(scale=np.sqrt(sigma_eta2), size=total)
    gamma = rng.normal(scale=np.sqrt(sigma_gamma2), size=total)
    #X[0], Y[0] = 0.0, 0.0

    for t in range(1, total):
        X[t] = b_t[t] * X[t-1] + eta[t]
        idx = t - tau_true
        x_lag = X[idx] if idx >= 0 else 0.0
        Y[t] = a_t[t] * Y[t-1] + c * x_lag + gamma[t]

    # return last N samples (drop burn and any earlier prepended samples)
    return X[-N:], Y[-N:], a_t[-N:], b_t[-N:]

# -- Windowed alpha estimation
def window_alpha_estimates_tau(Y, X, tau_assumed, window_size=70, step=1):
    """
    Slide a window across Y and X, estimate beta in each window by OLS regressing:
      Y_t = alpha * Y_{t-1} + beta * X_{t - tau_assumed}
    Returns:
      alphas: array of alpha estimates (one per window)
      center_points: array of time indices (center of each window)
    """
    n = len(Y)
    alphas = []
    center_points = []
    for start in range(0, n - window_size + 1, step):
        end = start + window_size
        Yw = Y[start:end]
        Xw = X[start:end]
        rows = []
        Ys = []
        maxlag = max(1, tau_assumed)
        # Build regression rows inside window, aligning indices so Yw[i] corresponds to time t
        for i in range(maxlag, window_size): # start from the first index where lags are defined
            y_t = Yw[i]
            y_lag = Yw[i-1]
            x_lag = Xw[i - tau_assumed]
            rows.append([y_lag, x_lag])
            Ys.append(y_t)
        A = np.asarray(rows, dtype=float)
        Yvec = np.asarray(Ys, dtype=float)
        reg = LinearRegression(fit_intercept=True).fit(A, Yvec)
        alpha = reg.coef_[0]
        alphas.append(float(alpha))
        center_points.append(start + window_size // 2)
    return np.array(alphas), np.array(center_points)

# -- Compute bias matrix over grid of (tau_true, tau_assumed)
def compute_bias_matrix_tau(cfg):
    """
    Compute bias matrices and also return simulation outputs organized by (tau_true, tau_assumed).
    
    Returns:
      bias_matrix: (len(tau_assumed_vals), len(tau_true_vals)) array of percent bias
      stderr_matrix: same shape, percent stderr (from Monte-Carlo)
      tau_assumed_vals: 1d-array of tau_assumed values (rows)
      tau_true_vals: 1d-array of tau_true values (cols)
      sims_dict: dictionary mapping (tau_true, tau_assumed) -> list of simulation dicts. Each simulation dict contains:
          - 'X': simulated X array (length N)
          - 'Y': simulated Y array (length N)
          - 'a_t': array of a(t)
          - 'b_t': array of b(t)
          - 'alphas': array of windowed alpha estimates for this simulation
          - 'center_points': array of corresponding window-centers for alpha estimates
          - 'slope': linear trend slope fitted to alphas (or np.nan)
    """
    # unpack config
    a0 = cfg['a0']; a_end = cfg['a_end']
    b0 = cfg['b0']; b_end = cfg['b_end']
    c = cfg['c']
    sigma_eta2 = cfg['sigma_eta2']; sigma_gamma2 = cfg['sigma_gamma2']
    N = cfg['N']; window_size = cfg['window_size']; step = cfg['step']
    n_sims = cfg['n_sims']; burn = cfg['burn']
    tau_true_vals = cfg['tau_true_vals']; tau_assumed_vals = cfg['tau_assumed_vals']
    true_a_trend = (a_end - a0) / (N - 1)
    # prepare outputs
    bias_matrix = np.full((len(tau_assumed_vals), len(tau_true_vals)), np.nan)
    stderr_matrix = np.full_like(bias_matrix, np.nan, dtype=float)
    sims_dict = {}  # keys: (tau_true, tau_assumed) -> list of per-sim dicts
    total_cells = len(tau_assumed_vals) * len(tau_true_vals)
    cell_idx = 0
    # iterate grid
    for i_tau_assumed, tau_assumed in enumerate(tau_assumed_vals):
        for j_tau_true, tau_true in enumerate(tau_true_vals):
            cell_idx += 1
            slope_estimates = []
            sims_for_cell = []  # accumulate simulation outputs for this (tau_true, tau_assumed)
            for sim in range(n_sims):
                seed = sim
                Xs, Ys, at, bt = simulate_time_varying_tau(a0, a_end, b0, b_end,
                                                       c, sigma_eta2, sigma_gamma2,
                                                       tau_true, N, burn=burn, seed=seed)
                alphas, center_points = window_alpha_estimates_tau(Ys, Xs, tau_assumed, window_size=window_size, step=step)
                # fit slope if enough windows
                mask = ~np.isnan(alphas)

                coeffs = np.polyfit(center_points[mask], alphas[mask], 1)
                slope = coeffs[0]
                slope_estimates.append(slope)
                # store simulation outputs in a dict
                sim_rec = {
                    'X': Xs,
                    'Y': Ys,
                    'a_t': at,
                    'b_t': bt,
                    'alphas': alphas,
                    'center_points': center_points,
                    'slope': slope
                }
                sims_for_cell.append(sim_rec)
            # compute summary statistics for this cell
            slope_estimates = np.array(slope_estimates)
            mean_slope = np.nanmean(slope_estimates)
            std_slope = np.nanstd(slope_estimates, ddof=1)
            bias = mean_slope - true_a_trend
            pct_bias = 100.0 * bias / true_a_trend if true_a_trend != 0 else np.nan
            bias_matrix[i_tau_assumed, j_tau_true] = pct_bias
            stderr_matrix[i_tau_assumed, j_tau_true] = 100.0 * std_slope / true_a_trend if true_a_trend != 0 else np.nan
            sims_dict[(int(tau_true), int(tau_assumed))] = sims_for_cell
    return bias_matrix, stderr_matrix, np.array(tau_assumed_vals), np.array(tau_true_vals), sims_dict

# -- Plotting function for bias heatmap
def plotly_heatmap_tau(bias_matrix, tau_assumed_vals, tau_true_vals):
    text = [[(f"{val:.1f}%" if (not np.isnan(val)) else "nan") for val in row] for row in bias_matrix]
    
    abs_max = np.nanmax(np.abs(bias_matrix))
    fig = go.Figure(data=go.Heatmap(
        z=bias_matrix, x=list(tau_true_vals), y=list(tau_assumed_vals),
        text=text, texttemplate="%{text}", colorscale='RdBu_r',
        zmin=-abs_max, zmax=abs_max, colorbar=dict(title='Bias (%)')
    ))
    fig.update_layout(xaxis_title=r'τ_true', yaxis_title=r'τ_assumed',
                      autosize=True, width=620, height=600)
    return fig

In [3]:
CONFIG = {
    'a0': 0.1, 'a_end': 0.6,        # a(t) evolves linearly from a0 -> a_end
    'b0': 0.12, 'b_end': 0.58,         # b(t) evolves linearly from b0 -> b_end
    'c': 0.5,
    'sigma_eta2': 0.5,
    'sigma_gamma2': 0.5,
    'N': 1000,                      # total length of retained series (large => slower true trend)
    'window_size': 100,              # stationary window size
    'step': 1,                     # sliding step for windows
    'n_sims': 200,                    # monte-carlo replicates (increase for precision)
    'tau_true_vals': list(range(0,6)),  # 0..5
    'tau_assumed_vals': list(range(0,6)),   # 0..5
    'burn': 10,
    }

In [4]:
bias_matrix, stderr_matrix, tau_assumed_vals, tau_true_vals, sims_dict = compute_bias_matrix_tau(CONFIG)

In [5]:
fig_wrong_lag_no_confounder = plotly_heatmap_tau(bias_matrix, tau_assumed_vals, tau_true_vals)

In [6]:
fig_wrong_lag_no_confounder.show()

In [ ]:
# fig_wrong_lag_no_confounder.write_image("fig_wrong_lag_no_confounder.svg")

## 2.Right lag, latent confounder


X and Y are coupled AR(1) with a latent confounder that induces cross-time covariance between noise terms.

This version lets both a(t) and b(t) vary linearly in time:

$X_t = b(t) X_{t-1} + \eta_t$

$Y_t = a(t) Y_{t-1} + c X_{t-tau} + \gamma_t$

$a(t) = a0 + s_a t$

$b(t) = b0 + s_b t$

Latent confounder $Z_t$ is AR(1): $Z_t = \phi Z_{t-1} + \epsilon_t$

$\eta_t = \sigma_{\eta} z_{1t} + p u_t$

$\gamma_t = \sigma_{\gamma} z_{2t} + q u_t$

In [ ]:
# -- Simulation function with latent confounder
def simulate_time_varying_confounder(a0, a_end, b0, b_end, c,
                                     p, q, phi, tau_true, N, burn=0, seed=0):
    """
    Simulate one realization of the time-varying AR(1) coupled system with a latent confounder.
    - p is the coefficient of Z_t in eta_t
    - q is the coefficient of Z_t in gamma_t
    Returns last N samples for X and Y and the corresponding a_t, b_t (length N).
    """
    rng = np.random.default_rng(seed)
    total = N + burn + abs(tau_true)
    # slopes for a and b over the returned N-block
    slope_a = (a_end - a0) / float(max(1, N - 1))
    slope_b = (b_end - b0) / float(max(1, N - 1))

    # build last-N linear sequences
    a_ret = a0 + slope_a * np.arange(N)
    b_ret = b0 + slope_b * np.arange(N)

    # prepend earlier values so full length == total
    pre_len = total - N
    a_pre = a0 - slope_a * np.arange(pre_len, 0, -1) if pre_len > 0 else np.array([])
    b_pre = b0 - slope_b * np.arange(pre_len, 0, -1) if pre_len > 0 else np.array([])
    a_t = np.concatenate([a_pre, a_ret]) if pre_len > 0 else a_ret.copy()
    b_t = np.concatenate([b_pre, b_ret]) if pre_len > 0 else b_ret.copy()

    # simulate latent Z (AR(1)) with white noise ~ N(0,1)
    eps = rng.normal(size=total)
    Z = np.zeros(total)
    for t in range(1, total):
        Z[t] = phi * Z[t-1] + eps[t]

    eta_tilde = rng.normal(size=total)
    gamma_tilde = rng.normal(size=total)
    eta = np.sqrt(1 - p**2) * eta_tilde + p * Z
    gamma = np.sqrt(1 - q**2) * gamma_tilde + q * Z

    X = np.zeros(total)
    Y = np.zeros(total)

    for t in range(1, total):
        X[t] = b_t[t] * X[t-1] + eta[t]
        Y[t] = a_t[t] * Y[t-1] + c * X[t - tau_true] + gamma[t]

    # return last N samples
    return X[-N:], Y[-N:], a_t[-N:], b_t[-N:]

# -- Windowed alpha estimation with confounder
def window_alpha_estimates_confounder(Y, X, tau, window_size, step=1):
    """
    Sliding-window OLS estimates of alpha in regression
      Y_t ~ alpha * Y_{t-1} + beta * X_{t - tau}
    Returns:
      alphas: array of estimated alpha for each window (np.nan if regression fails)
      center_points: array of center indices (integers) of each window
    """
    N = len(X)
    center_points = []
    alphas = []
    for start in range(0, N - window_size + 1, step):
        end = start + window_size
        Yw = Y[start:end]
        Xw = X[start:end]
        rows = []
        Ys = []
        # build rows: index i corresponds to time t = start + i
        for i in range(1, window_size):  # y_{t-1} defined for i>=1
            # check X lag availability within the window
            if (i - tau) < 0:
                continue
            y_t = Yw[i]
            y_lag = Yw[i-1]
            x_lag = Xw[i - tau]
            rows.append([y_lag, x_lag])
            Ys.append(y_t)

        A = np.asarray(rows, dtype=float)
        Yvec = np.asarray(Ys, dtype=float)
        reg = LinearRegression(fit_intercept=True).fit(A, Yvec)
        alpha_est = float(reg.coef_[0])  # coefficient for y_{t-1}
        alphas.append(alpha_est)
        center_points.append(start + window_size // 2)

    return np.array(alphas, dtype=float), np.array(center_points, dtype=float)

# -- Compute bias matrix over grid of p and q for fixed phi
def compute_bias_matrix_confounder(cfg):
    """
    Compute bias matrix over grid of p and q for a fixed phi.
    cfg expects keys:
      - a0, a_end, b0, b_end
      - c
      - N, window_size, step
      - n_sims, burn
      - tau_true (int)
      - p_vals (iterable)
      - q_vals (iterable)
      - phi (float)  # fixed
    Returns:
      bias_matrix (shape len(q_vals) x len(p_vals)) in percent,
      stderr_matrix (same shape) in percent,
      np.array(q_vals), np.array(p_vals), sims_dict
    """
    a0 = float(cfg['a0']); a_end = float(cfg['a_end'])
    b0 = float(cfg['b0']); b_end = float(cfg['b_end'])
    c = float(cfg['c'])
    N = int(cfg['N']); window_size = int(cfg['window_size']); step = int(cfg.get('step', 1))
    n_sims = int(cfg['n_sims']); burn = int(cfg.get('burn', 0))
    tau_true = int(cfg['tau_true'])
    p_vals = list(cfg['p_vals'])
    q_vals = list(cfg['q_vals'])
    phi = float(cfg['phi'])

    true_a_trend = (a_end - a0) / float(N - 1)

    bias_matrix = np.full((len(q_vals), len(p_vals)), np.nan, dtype=float)
    stderr_matrix = np.full_like(bias_matrix, np.nan, dtype=float)
    sims_dict = {}

    total_cells = len(q_vals) * len(p_vals)
    cell_idx = 0

    for i_q, q in enumerate(q_vals):
        for j_p, p in enumerate(p_vals):
            cell_idx += 1
            slopes = []
            sims_for_cell = []
            for sim_idx in range(n_sims):
                seed = sim_idx + i_q * 10 + j_p * 100 +1 
                Xs, Ys, at, bt = simulate_time_varying_confounder(
                    a0, a_end, b0, b_end, c,
                    float(p), float(q), float(phi), int(tau_true),
                    N, burn=burn, seed=seed
                )
                alphas, centers = window_alpha_estimates_confounder(Ys, Xs, int(tau_true), window_size=window_size, step=step)
                mask = ~np.isnan(alphas)

                coeffs = np.polyfit(centers[mask], alphas[mask], 1)
                slope = float(coeffs[0])
                slopes.append(slope)
                sim_rec = {
                    'X': Xs,
                    'Y': Ys,
                    'a_t': at,
                    'b_t': bt,
                    'alphas': alphas,
                    'center_points': centers,
                    'slope': slope
                }
                sims_for_cell.append(sim_rec)
            sims_dict[(float(p), float(q))] = sims_for_cell
            slopes_arr = np.array(slopes, dtype=float)
            if slopes_arr.size == 0:
                mean_slope = np.nan
                std_slope = np.nan
            else:
                mean_slope = float(np.nanmean(slopes_arr))
                std_slope = float(np.nanstd(slopes_arr, ddof=(1 if slopes_arr.size>1 else 0)))
            bias = mean_slope - true_a_trend
            pct_bias = 100.0 * bias / true_a_trend if true_a_trend != 0 else np.nan
            pct_stderr = 100.0 * std_slope / true_a_trend if true_a_trend != 0 else np.nan
            bias_matrix[i_q, j_p] = pct_bias
            stderr_matrix[i_q, j_p] = pct_stderr

    return bias_matrix, stderr_matrix, np.array(q_vals), np.array(p_vals), sims_dict

# -- Plotting function for bias heatmap with confounder
def plot_bias_heatmap_confounder(bias_matrix, p_vals, q_vals):    
    text = [[(f"{val:.1f}%" if (not np.isnan(val)) else "nan") for val in row] for row in bias_matrix]
    abs_max = np.nanmax(np.abs(bias_matrix))

    x_labels = [f"{v:.1f}" for v in p_vals]
    y_labels = [f"{v:.1f}" for v in q_vals]

    fig = go.Figure(data=go.Heatmap(
        z=bias_matrix, x=x_labels, y=y_labels,
        text=text, texttemplate="%{text}",
        colorscale="RdBu_r", colorbar=dict(title="Bias (%)"),
        zmin=-abs_max, zmax=abs_max
    ))

    fig.update_layout(
        xaxis_title="p", yaxis_title="q",
        xaxis=dict(type="category"), yaxis=dict(type="category"),
        width=500, height=500, template="plotly_white", autosize=True)
    return fig

In [9]:
config = {
    'a0': 0.1, 'a_end': 0.6,
    'b0': 0.12, 'b_end': 0.58,
    'c': 0.5, 'phi': 0.5,
    'N': 1000, 'window_size': 100, 'step': 1,
    'n_sims': 200, 'burn': 10,
    'tau_true': 1,
    'p_vals': [-0.9, -0.5, 0.0, 0.5, 0.9],
    'q_vals': [-0.9, -0.5, 0.0, 0.5, 0.9],
}

In [10]:
bias_matrix, stderr_matrix, p_vals, q_vals, sims_dict = compute_bias_matrix_confounder(config)

In [11]:
fig_right_lag_latent_confounder = plot_bias_heatmap_confounder(bias_matrix, p_vals, q_vals)

In [12]:
fig_right_lag_latent_confounder.show()

In [ ]:
# fig_right_lag_latent_confounder.write_image("fig_right_lag_latent_confounder.svg")